In [13]:
import pandas as pd
import numpy as np

# Load the gene expression data and GPL mapping data
gene_df = pd.read_csv('Raw.csv')
gpl_df = pd.read_csv('GPL.csv')

# Convert the 'Symbol' and 'Name' columns to lower case
gene_df['Symbol'] = gene_df['Symbol'].str.lower()
gpl_df['Symbol v12'] = gpl_df['Symbol v12'].str.lower()
gpl_df['Name'] = gpl_df['Name'].str.lower()

# Assuming 'numeric_columns' is a list of column names that you want to normalize
numeric_columns = gene_df.columns.difference(['Symbol'])

# Add a small constant to the numeric columns in gene_df to avoid division by zero
constant = 1
gene_df[numeric_columns] = gene_df[numeric_columns].add(constant)

# Apply log2 normalization to the numeric columns in gene_df
gene_df[numeric_columns] = np.log2(gene_df[numeric_columns])

# Drop the invalid and null rows in the data set
gene_df = gene_df.dropna()

# Merge the gene expression data and GPL mapping data after converting 'Symbol' columns to lowercase
merged_df = gene_df.merge(gpl_df, left_on='Symbol', right_on='Symbol v12', how='left')

# Drop rows where 'Symbol v12' is null
merged_df = merged_df.dropna(subset=['Symbol v12'])

# Group the merged data by 'Symbol v12' and calculate the median for each group
median_df = merged_df.groupby('Symbol v12').median(numeric_only=True)

# Resulting dataframe
print(median_df)

                      GSM577963_CTRL_01  GSM577964_CTRL_02  GSM577965_CTRL_03  \
Symbol v12                                                                      
15 kda selenoprotein           8.758223           7.826548           8.430453   
15e1.2                         7.219169           7.629357           7.629357   
2'-pde                         7.383704           7.238405           6.794416   
3.8-1                          7.276124           7.139551           7.129283   
5-ht3c2                       10.328675           9.071462           9.368506   
...                                 ...                ...                ...   
zyg11b                         8.475733           8.308339           7.499846   
zyg11bl                        7.000000           7.076816           7.076816   
zyx                            9.879583           8.546894           8.426265   
zzef1                          7.607330           7.118941           7.238405   
zzz3                        

In [14]:
from scipy import stats

# Identify column names for each group
ctrl_columns = [col for col in median_df.columns if 'CTRL' in col]
ms_columns = [col for col in median_df.columns if 'RA' in col] # Assuming 'RA' represents MS based on your data snippet
t2d_columns = [col for col in median_df.columns if 'T2D' in col]
cad_columns = [col for col in median_df.columns if 'CAD' in col]

groups = {'MS': ms_columns, 'T2D': t2d_columns, 'CAD': cad_columns}

# For each group, perform the t-tests and fold change calculation
for disease, disease_columns in groups.items():
    # Perform a two-sample t-test for each gene
    t_test_results = stats.ttest_ind(median_df[ctrl_columns], median_df[disease_columns], axis=1)

    # Create a DataFrame from the t-test results
    t_test_df = pd.DataFrame({'statistic': t_test_results.statistic, 'pvalue': t_test_results.pvalue}, index=median_df.index)

    # Filter out genes with p > 0.05
    significant_genes = t_test_df[t_test_df['pvalue'] <= 0.05]

    # Calculate the mean log2 expression in each group for each gene
    ctrl_means = median_df[ctrl_columns].mean(axis=1)
    disease_means = median_df[disease_columns].mean(axis=1)

    # Calculate the absolute fold change for each gene
    fc = abs(ctrl_means - disease_means)

    # Filter genes based on fold change
    de_genes = significant_genes[fc >= 1]

    # Print the number of differentially expressed genes for this disease
    print(f'For {disease}, the number of DEGs is {len(de_genes)}')

For MS, the number of DEGs is 3294
For T2D, the number of DEGs is 809
For CAD, the number of DEGs is 1983


C:\Users\disha\AppData\Local\Temp\ipykernel_19572\2803163799.py:14: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  t_test_results = stats.ttest_ind(median_df[ctrl_columns], median_df[disease_columns], axis=1)
C:\Users\disha\AppData\Local\Temp\ipykernel_19572\2803163799.py:30: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  de_genes = significant_genes[fc >= 1]
C:\Users\disha\AppData\Local\Temp\ipykernel_19572\2803163799.py:14: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  t_test_results = stats.ttest_ind(median_df[ctrl_columns], median_df[disease_columns], axis=1)
C:\Users\disha\AppData\Local\Temp\ipykernel_19572\2803163799.py:30: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  